# Denmark

In [13]:
import pandas as pd
import geopandas as gpd

pd.options.mode.chained_assignment = None

## Fetch results

In [23]:
df = pd.read_csv('../../data/raw/dnk.csv', sep=';', encoding='latin1', skiprows=1)

In [24]:
df.head()

,,.1,VALID BALLOTS,PERSONAL BALLOTS,VALID ABSENTEE BALLOTS,INVALID BALLOTS,INVALID BALLOTS CAST ON POLLING STATION,Blanks,INVALID ABSENTEE BALLOTS,Blanks.1,NUMBER OF VOTERS
0,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,Copenhagen,219140.0,145524.0,18511.0,6347.0,5851.0,4783.0,496.0,217.0,400933.0
2,,Frederiksberg,46087.0,32153.0,5406.0,992.0,868.0,781.0,124.0,69.0,74849.0
3,,Dragør,6088.0,4894.0,543.0,105.0,88.0,73.0,17.0,14.0,10208.0
4,,Tårnby,15990.0,12070.0,1178.0,361.0,324.0,286.0,37.0,19.0,30709.0


In [25]:
(df['VALID BALLOTS'].sum() + df['INVALID BALLOTS'].sum()) / df['NUMBER OF VOTERS'].sum()

0.5631566581645651

In [26]:
df['turnout'] = (df['VALID BALLOTS'] + df['INVALID BALLOTS']) / df['NUMBER OF VOTERS']
df = df.iloc[1:]
df = df.rename(columns={' .1': 'name'})
df = df[['name', 'turnout']]

In [28]:
df.turnout = df.turnout * 100

## Merge

In [29]:
geo = gpd.read_file('../../data/shapefiles/dnk/gadm36_DNK_2.shp')

In [30]:
df.name = df.name.replace('Copenhagen', 'København')
df.name = df.name.replace('Høje-Taastrup', 'Høje Taastrup')
df.name = df.name.replace('Aarhus', 'Århus')
df.name = df.name.replace('Vesthimmerlands', 'Vesthimmerland')

In [31]:
geo = geo.merge(df, left_on='NAME_2', right_on='name', how='left')

## Export

In [32]:
geo = geo[['name', 'turnout', 'geometry']]

In [33]:
geo['country'] = 'dnk'

In [35]:
geo.to_file('../../data/processed/dnk', encoding='utf8')